In [ ]:
# Global parameters

PACKETSIZE = 100

IP = '192.168.100.10'

PORT = 10000

TICKS_1000MS = 60000

In [ ]:
# Imports

import time
import socket
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import zlib
import os
import folium
import io
from PIL import Image
import selenium
import asyncio
import winsdk.windows.devices.geolocation as wdg
import keyboard
import matplotlib.pyplot as plt
import subprocess
from geopy.distance import geodesic

In [ ]:
# Get coordinates

base_coords = [0,0]

async def getCoords():
    locator = wdg.Geolocator()
    try:
        pos = await locator.get_geoposition_async()
        return [pos.coordinate.latitude, pos.coordinate.longitude]
    except PermissionError:
        print("Permission error : Location services are disabled")

base_coords = await getCoords()

print(f'Base coordinates : {base_coords[0],base_coords[1]}')

In [ ]:
# Enable TCP listener

#IP = socket.gethostbyname(socket.gethostname())

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

sock.bind((IP,PORT))

sock.listen()

print('Listening on %s:%d' % (IP,PORT))

conn, addr = sock.accept()

print('Receiving from %s:%d' % (addr[0],addr[1]))

conn.settimeout(5)

In [ ]:
# Read initial response

running = True

while running:
    try:
        data = conn.recv(100)
        print(data.decode()[:-1])
    except socket.timeout:
        running = False

In [ ]:
# Start streaming

sent = conn.send(bytes('start\n','utf-8'))

In [ ]:
# Read packets

stats_packetbytes = np.empty(0).astype(int)
stats_start = np.empty(0)
stats_dur = np.empty(0)
stats_sysdatetime = np.empty(0)
stats_crc32 = np.empty(0).astype(int)
stats_crc32_calc = np.empty(0).astype(int)

stats_lat = np.empty(0).astype(float)
stats_lng = np.empty(0).astype(float)
stats_alt = np.empty(0).astype(int)
stats_vel = np.empty(0).astype(int)
stats_rssi = np.empty(0).astype(int)

stats_ID = np.empty(0).astype(str)
stats_num = np.empty(0).astype(int)

stats_dist = np.empty(0).astype(int)

print('Press ESC to stop')

running = True

while running:
    
    try:
        
        packet = conn.recv(10 + 12 + 4 + 8)
        
        stats_sysdatetime = np.append(stats_sysdatetime,datetime.now())
        
        start = datetime.fromtimestamp( int.from_bytes(packet[0:4], 'big') ) + timedelta( seconds = int.from_bytes(packet[4:6], 'big') / TICKS_1000MS )

        dur = timedelta( seconds = int.from_bytes(packet[6:10], 'big') / TICKS_1000MS )
              
        lat = int.from_bytes(packet[-24:-20], 'big')/1e6
        lng = int.from_bytes(packet[-20:-16], 'big')/1e6
        alt = int.from_bytes(packet[-16:-14], 'big')
        vel = int(packet[-14])
        rssi = int(packet[-13])
        
        # received checksum
        
        crc32 = int.from_bytes(packet[-12:-8], 'big')
        
        # calculated checksum
        
        crc32_calc = zlib.crc32(packet[:-12]) & 0xffffffff
        
        # ID
        
        ID = packet[-8:-2].decode('utf-8')
        
        # packet number (packetID)
        
        num = int.from_bytes(packet[-2:], 'big')
        
        # calculate distance in meters
        
        dist = round(geodesic(base_coords,[lat,lng]).meters)

        # print message
        
        s = 'Device : %s | Packet : %04d | Datetime : %s | CRC32 : %08X / %08X | RSSI : %d dBm | Distance : %d m' % (ID,num,start.strftime('%d/%m/%Y - %H:%M:%S.%f'),crc32,crc32_calc,-rssi,dist)
        
        print(s, end = '\r', flush = True)
        
        # append to total arrays
        
        stats_start = np.append(stats_start,start)
        stats_packetbytes = np.append(stats_packetbytes,(len(packet)))
        stats_dur = np.append(stats_dur,dur)
            
        stats_lat = np.append(stats_lat, lat)           
        stats_lng = np.append(stats_lng, lng)
        stats_alt = np.append(stats_alt, alt)           
        stats_vel = np.append(stats_vel, vel) 
        stats_rssi = np.append(stats_rssi, -rssi)
        
        stats_crc32 = np.append(stats_crc32, crc32)
        stats_crc32_calc = np.append(stats_crc32_calc, crc32_calc)
        
        stats_ID = np.append(stats_ID, ID) 
        stats_num = np.append(stats_num, num)
        
        stats_dist = np.append(stats_dist, dist)
        
        if keyboard.is_pressed('esc'):
            running = False
  
    except socket.timeout:
        
        running = False

In [ ]:
# Stop streaming

sent = conn.send(bytes('stop\n','utf-8'))

time.sleep(1)

conn.close()
sock.close()

In [ ]:
# Packet stats

stats_start_date = np.array([t.strftime('%d/%m/%Y') for t in stats_start])
stats_start_time = np.array([t.strftime('%H:%M:%S.%f') for t in stats_start])
stats_systime = np.array([t.strftime('%H:%M:%S.%f') for t in stats_sysdatetime])
stats_dur_secs = np.array([t.total_seconds() for t in stats_dur])

# Packet date
df = pd.DataFrame(stats_start_date, columns = ['Packet start date'])
# Packet time
df['Packet start time'] = stats_start_time
# Packet length
df['Packet bytes'] = stats_packetbytes
# Packet duration (as received)
df['Packet duration (ms)'] = 1000*stats_dur_secs
# System time when packet is received
df['PC time'] = stats_systime
# Time difference between system time and packet time
df['Time diff (ms)'] = 1000*np.array([t.total_seconds() for t in stats_sysdatetime - stats_start])
# lat
df['Latitude'] = stats_lat
# lng
df['Longitude'] = stats_lng
# altitude
df['Altitude'] = stats_alt
# velocitu
df['Velocity'] = stats_vel
# rssi
df['RSSI (dBm)'] = stats_rssi
# distance
df['Distance (m)'] = stats_dist
# Packet CRC32
df['CRC32 received'] = stats_crc32
# Packet CRC32 calculated
df['CRC32 calculated'] = stats_crc32_calc
# check
df['Check'] = np.where(df['CRC32 received'] == df['CRC32 calculated'], 'Ok', 'Corrupt')

# Formatting

df['Packet duration (ms)'] = df['Packet duration (ms)'].apply(lambda x: '{:.2f}'.format(x))
df['Time diff (ms)'] = df['Time diff (ms)'].apply(lambda x: '{:.2f}'.format(x))
df['CRC32 received'] = df['CRC32 received'].apply(lambda x: '{:08X}'.format(x))
df['CRC32 calculated'] = df['CRC32 calculated'].apply(lambda x: '{:08X}'.format(x))

# Save

DT = datetime.now().strftime('%Y_%d_%m_%H%M')
df.to_csv(f'Packets_B_{DT}.txt', sep = '\t', index = False)

In [ ]:
#df = pd.read_csv('Packets_B_', sep='\t')

def get_color(rssi):
    
    cmap = plt.get_cmap('jet')
    norm = plt.Normalize(30,100)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    rgba_color = sm.to_rgba(-rssi)
    return "#{:02x}{:02x}{:02x}".format(int(rgba_color[0]*255), int(rgba_color[1]*255), int(rgba_color[2]*255))

m = folium.Map(location=[np.mean(df['Latitude']), np.mean(df['Longitude'])], zoom_start=19)

folium.CircleMarker(location=base_coords, radius=10, color="black", weight=2.0, fill=True, fill_color='white', fill_opacity=1.0, popup=f'Base').add_to(m)

for lat, lon, r, d in zip(df['Latitude'], df['Longitude'], df['RSSI (dBm)'], df['Distance (m)']):

    folium.CircleMarker(location=(lat, lon), radius=10, color="black", weight=1.0, fill=True, fill_color=get_color(r), fill_opacity=1.0, popup=f'RSSI\n{r} dBm\nDistance\n{d} m').add_to(m)

DT = datetime.now().strftime('%Y_%d_%m_%H%M')
m.save(f'Map_{DT}.html')

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(f'Map_{DT}.png')

# Show map

m